In [1]:
import pandas as pd
import re
import string
import matplotlib.pyplot as plt
import numpy as np
import spacy
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer


In /Users/amybeisel/opt/anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The savefig.frameon rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In /Users/amybeisel/opt/anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The verbose.level rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In /Users/amybeisel/opt/anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The verbose.fileo rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.


In [2]:
df = pd.read_csv('data/cannabis.csv')

In [3]:
print(df.shape)
df


(2351, 6)


,Strain,Type,Rating,Effects,Flavor,Description
0,100-Og,hybrid,4.0,"Creative,Energetic,Tingly,Euphoric,Relaxed","Earthy,Sweet,Citrus",$100 OG is a 50/50 hybrid strain that packs a ...
1,98-White-Widow,hybrid,4.7,"Relaxed,Aroused,Creative,Happy,Energetic","Flowery,Violet,Diesel",The ‘98 Aloha White Widow is an especially pot...
2,1024,sativa,4.4,"Uplifted,Happy,Relaxed,Energetic,Creative","Spicy/Herbal,Sage,Woody",1024 is a sativa-dominant hybrid bred in Spain...
3,13-Dawgs,hybrid,4.2,"Tingly,Creative,Hungry,Relaxed,Uplifted","Apricot,Citrus,Grapefruit",13 Dawgs is a hybrid of G13 and Chemdawg genet...
4,24K-Gold,hybrid,4.6,"Happy,Relaxed,Euphoric,Uplifted,Talkative","Citrus,Earthy,Orange","Also known as Kosher Tangie, 24k Gold is a 60%..."
...,...,...,...,...,...,...
2346,Zeus-Og,hybrid,4.7,"Happy,Uplifted,Relaxed,Euphoric,Energetic","Earthy,Woody,Pine",Zeus OG is a hybrid cross between Pineapple OG...
2347,Zkittlez,indica,4.6,"Relaxed,Happy,Euphoric,Uplifted,Sleepy","Sweet,Berry,Grape",Zkittlez is an indica-dominant mix of Grape Ap...
2348,Zombie-Kush,indica,5.0,"Relaxed,Sleepy,Talkative,Euphoric,Happy","Earthy,Sweet,Spicy/Herbal",Zombie Kush by Ripper Seeds comes from two dif...
2349,Zombie-Og,indica,4.4,"Relaxed,Sleepy,Euphoric,Happy,Hungry","Sweet,Earthy,Pungent",If you’re looking to transform into a flesh-ea...


In [4]:
df['Effects'].isnull().sum()

0

In [5]:
df['Description'].isnull().value_counts()

False    2318
True       33
Name: Description, dtype: int64

In [6]:
df[df['Description'].isnull()]

,Strain,Type,Rating,Effects,Flavor,Description
163,Banana-Candy,indica,4.2,"Relaxed,Euphoric,Uplifted,Creative,Happy","Tree,Fruit,Earthy,Sweet",NaN
368,Boysenberry,hybrid,3.6,"Giggly,Relaxed,Talkative,Energetic,Euphoric","Berry,Sweet,Earthy",NaN
594,Crater-Lake,hybrid,4.6,"Happy,Uplifted,Talkative,Relaxed,Creative","Citrus,Pine,Pungent",NaN
679,Domino,indica,4.0,"Sleepy,Happy,Tingly,Uplifted,Euphoric",NaN,NaN
736,Earthquake,hybrid,4.5,"Creative,Euphoric,Uplifted,Happy,Energetic","Woody,Pine,Earthy",NaN
817,Frosty,indica,4.4,"Sleepy,Relaxed,Creative,Hungry,Happy","Earthy,Sweet,Skunk",NaN
856,Ghost-Bubba,hybrid,4.6,"Happy,Energetic,Uplifted,Relaxed,Giggly","Sweet,Pungent,Grape",NaN
899,Goldwing,sativa,4.7,"Uplifted,Happy,Euphoric,Creative,Energetic",NaN,NaN
949,Green-Hornet,hybrid,4.2,"Euphoric,Uplifted,Relaxed,Giggly,Happy","Sweet,Woody,Pine",NaN
955,Green-Monster,hybrid,3.7,"Happy,Relaxed,Uplifted,Hungry,Giggly","Earthy,Chemical,Tar",NaN


In [7]:
df = df.dropna()

In [8]:
len(df['Description'])

2277

In [9]:
df.shape

(2277, 6)

In [10]:
df["Description"].dtype

dtype('O')

In [11]:
#df['Description'] = df['Description'].apply(str).str.replace('.', ',')

In [12]:
df["Effects"][0]

'Creative,Energetic,Tingly,Euphoric,Relaxed'

In [13]:
import spacy
from spacy.tokenizer import Tokenizer
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer

nlp = spacy.load("en_core_web_lg")
df.head()

# The Tokenizer
tokenizer = Tokenizer(nlp.vocab)

# Make the tokens for description
description_tokens = []
for txt in tokenizer.pipe(df['Description'], batch_size=500):
    txt_tokens = [token.text for token in txt]
    description_tokens.append(txt_tokens)
df['description_tokens'] = description_tokens
print(df['description_tokens'].head())


# # Make the tokens for flavor
# flavor_tokens = []
# for txt in tokenizer.pipe(df['Flavor'], batch_size=500):
#     txt_tokens = [token.text for token in txt]
#     flavor_tokens.append(txt_tokens)
# df['flavor_tokens'] = flavor_tokens
# print(df['flavor_tokens'].head())



# # Make the tokens for effects
# effects_tokens = []
# for txt in tokenizer.pipe(df['Effects'], batch_size=500):
#     txt_tokens = [token.text for token in txt]
#     effects_tokens.append(txt_tokens)
# df['effects_tokens'] = effects_tokens
# print(df['effects_tokens'].head())

#df['combined'] = description_tokens + flavor_tokens + effects_tokens

#print(df['combined'] = description_tokens + flavor_tokens + effects_tokens)

0    [$100, OG, is, a, 50/50, hybrid, strain, that,...
1    [The, ‘98, Aloha, White, Widow, is, an, especi...
2    [1024, is, a, sativa-dominant, hybrid, bred, i...
3    [13, Dawgs, is, a, hybrid, of, G13, and, Chemd...
4    [Also, known, as, Kosher, Tangie,, 24k, Gold, ...
Name: description_tokens, dtype: object


/Users/amybeisel/opt/anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [14]:
df.head()

,Strain,Type,Rating,Effects,Flavor,Description,description_tokens
0,100-Og,hybrid,4.0,"Creative,Energetic,Tingly,Euphoric,Relaxed","Earthy,Sweet,Citrus",$100 OG is a 50/50 hybrid strain that packs a ...,"[$100, OG, is, a, 50/50, hybrid, strain, that,..."
1,98-White-Widow,hybrid,4.7,"Relaxed,Aroused,Creative,Happy,Energetic","Flowery,Violet,Diesel",The ‘98 Aloha White Widow is an especially pot...,"[The, ‘98, Aloha, White, Widow, is, an, especi..."
2,1024,sativa,4.4,"Uplifted,Happy,Relaxed,Energetic,Creative","Spicy/Herbal,Sage,Woody",1024 is a sativa-dominant hybrid bred in Spain...,"[1024, is, a, sativa-dominant, hybrid, bred, i..."
3,13-Dawgs,hybrid,4.2,"Tingly,Creative,Hungry,Relaxed,Uplifted","Apricot,Citrus,Grapefruit",13 Dawgs is a hybrid of G13 and Chemdawg genet...,"[13, Dawgs, is, a, hybrid, of, G13, and, Chemd..."
4,24K-Gold,hybrid,4.6,"Happy,Relaxed,Euphoric,Uplifted,Talkative","Citrus,Earthy,Orange","Also known as Kosher Tangie, 24k Gold is a 60%...","[Also, known, as, Kosher, Tangie,, 24k, Gold, ..."


In [15]:
df['test'] = df["Effects"] + ',' + df["Flavor"] 
df["test"][0]

/Users/amybeisel/opt/anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


'Creative,Energetic,Tingly,Euphoric,Relaxed,Earthy,Sweet,Citrus'

In [16]:
def tokenize(document):
    
    doc = nlp(document)
    
    return [token.lemma_.strip() for token in doc if (token.is_stop != True) and (token.is_punct != True)]

In [17]:

# Instantiate vectorizer object
tfidf = TfidfVectorizer(stop_words = 'english',
                       #ngram_range = (1,2),
                       max_features = 5000)

# Create a vocabulary and tf-idf score per document
dtm = tfidf.fit_transform(df['Description'])
                         

# Get feature names to use as dataframe column headers
dtm = pd.DataFrame(dtm.todense(), columns=tfidf.get_feature_names())
nn = NearestNeighbors(n_neighbors=4, algorithm='kd_tree')
nn.fit(dtm)


# View Feature Matrix as DataFrame
print(dtm.shape)
dtm.head()

(2277, 5000)


,09,10,100,1024,11,12,13,14,15,16,...,zelly,zen,zest,zestful,zesty,zeta,zion,zkittlez,zombie,zone
0,0.0,0.0,0.435383,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.000000,0.487428,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.000000,0.000000,0.0,0.0,0.435528,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
df['Description'][0][:150]

'$100 OG is a 50/50 hybrid strain that packs a strong punch. The name supposedly refers to both its strength and high price when it first started showi'

In [19]:
df['Description'][338][:150]

'A hybrid cross of Blueberry and Haze, provides a sweet smoke and pleasant high. \xa0Pace yourself with this one, overdoing it can send you to dreamland.\xa0'

In [20]:
ideal = ["""
Creative,Uplifted,Tingly,Euphoric,Relaxed, Giggly
"""]

In [21]:
# Query the ideal descprition
new = tfidf.transform(ideal)
new

<1x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 6 stored elements in Compressed Sparse Row format>

In [22]:
nn.kneighbors(new.todense())

(array([[1., 1., 1., 1.]]), array([[1605, 1606, 1698, 1607]]))

In [23]:
# most ideal description,strain and flavor
df['Description'][1605]

'Pineapple Super Silver Haze from Fire Bros. is a sativa strain that modifies the widely cherished Super Silver Haze with a Pineapple hybrid strain. This second parent is thought to be either Pineapple Express or Pineapple, a phenotype of Ed Rosenthal Super Bud. The Haze genetics in this strain come through in both flavor and effect, as Pineapple Super Silver Haze delivers a high-flying cerebral buzz alongside a spicy, zesty flavor. Its aroma is more of a tropical medley of candied mango, pineapple, and oranges. Like a cup of coffee, this sativa is a perfect pick-me-up with motivating, talkative, and creative effects and minimal heaviness in the body.'

In [24]:
df['Strain'][1605]

'Pineapple-Super-Silver-Haze'

In [25]:
df['Effects'][1605]

'Happy,Euphoric,Energetic,Focused,Tingly'

In [26]:
df['Flavor'][1605]

'Pineapple,Sweet,Earthy'

In [27]:
#to be pickled....
#nn, dtm, tfidf

In [28]:
import pickle
# Dump the trained classifier (nn)  with Pickle
pickle_filename = 'model.pkl'
pickled_model = open(pickle_filename, 'wb')  # Open the file to save as pkl file
pickle.dump(nn, pickled_model)
pickled_model.close() # Close the pickle instances


In [29]:
# Loading the saved model
model_pkl = open(pickle_filename, 'rb')
NN_model = pickle.load(model_pkl)
print ("Loaded model :: ", NN_model)  # print to verify

Loaded model ::  NearestNeighbors(algorithm='kd_tree', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=4, p=2,
                 radius=1.0)


In [30]:
import pickle
# Dump the trained classifier (nn)  with Pickle
pickle_filename_1 = 'tfidf.pkl'
pickled_model_1 = open(pickle_filename_1, 'wb')  # Open the file to save as pkl file
pickle.dump(tfidf, pickled_model_1)
pickled_model_1.close() # Close the pickle instances

In [31]:
# Loading the saved model
model_pkl_1 = open(pickle_filename_1, 'rb')
tfidf_model = pickle.load(model_pkl_1)
print ("Loaded model :: ", tfidf_model)  # print to verify

Loaded model ::  TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=5000,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words='english', strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)


In [32]:
dir(NN_model)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_check_algorithm_metric',
 '_fit',
 '_fit_X',
 '_fit_method',
 '_get_param_names',
 '_get_tags',
 '_kneighbors_reduce_func',
 '_more_tags',
 '_pairwise',
 '_radius_neighbors_reduce_func',
 '_tree',
 'algorithm',
 'effective_metric_',
 'effective_metric_params_',
 'fit',
 'get_params',
 'kneighbors',
 'kneighbors_graph',
 'leaf_size',
 'metric',
 'metric_params',
 'n_jobs',
 'n_neighbors',
 'p',
 'radius',
 'radius_neighbors',
 'radius_neighbors_graph',
 'set_params']

In [33]:
ideal2 = ["happy,Sleepy,Apricot,Citrus,Grapefruit"]

In [34]:
new2 = tfidf_model.transform(ideal2)

In [35]:
NN_model.kneighbors(new2.todense())

(array([[1., 1., 1., 1.]]), array([[1606, 1607, 1605, 1698]]))

In [36]:
df['Strain'][1606]

'Pineapple-Thai'

In [37]:
df['Flavor'][1606]

'Pineapple,Citrus,Sweet'